In [ ]:
### perform object detection using HSV in OpenCV:


import cv2
import numpy as np

# Define the callback function for the trackbar
def nothing(x):
    pass

# Create a named window for the trackbars
cv2.namedWindow('Trackbars')

# Create trackbars for Hue, Saturation, and Value
cv2.createTrackbar('Hue_Low', 'Trackbars', 0, 179, nothing)
cv2.createTrackbar('Hue_High', 'Trackbars', 179, 179, nothing)
cv2.createTrackbar('Saturation_Low', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('Saturation_High', 'Trackbars', 255, 255, nothing)
cv2.createTrackbar('Value_Low', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('Value_High', 'Trackbars', 255, 255, nothing)

# Load an image
img = cv2.imread('smarties.png')

# Convert the image to HSV color space
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

while True:
    # Get the current trackbar positions
    h_low = cv2.getTrackbarPos('Hue_Low', 'Trackbars')
    h_high = cv2.getTrackbarPos('Hue_High', 'Trackbars')
    s_low = cv2.getTrackbarPos('Saturation_Low', 'Trackbars')
    s_high = cv2.getTrackbarPos('Saturation_High', 'Trackbars')
    v_low = cv2.getTrackbarPos('Value_Low', 'Trackbars')
    v_high = cv2.getTrackbarPos('Value_High', 'Trackbars')

    # Define the lower and upper bounds of the color to detect
    lower_bound = np.array([h_low, s_low, v_low])
    upper_bound = np.array([h_high, s_high, v_high])

    # Create a mask using the lower and upper bounds
    mask = cv2.inRange(hsv, lower_bound, upper_bound)

    # Apply the mask to the original image
    result = cv2.bitwise_and(img, img, mask=mask)

    # Show the original image and the result
    cv2.imshow('Original', img)
    cv2.imshow('Result', result)

    # Wait for the user to press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cv2.destroyAllWindows()

 


In [ ]:
### perform object tracking using HSV in OpenCV:


import cv2
import numpy as np

def nothing(x):
    pass

# Load the video file
cap = cv2.VideoCapture('dog.mp4')

# Create a window and trackbars to adjust the color thresholds
cv2.namedWindow('Object Detection')
cv2.createTrackbar('Low H', 'Object Detection', 0, 179, nothing)
cv2.createTrackbar('High H', 'Object Detection', 179, 179, nothing)
cv2.createTrackbar('Low S', 'Object Detection', 0, 255, nothing)
cv2.createTrackbar('High S', 'Object Detection', 255, 255, nothing)
cv2.createTrackbar('Low V', 'Object Detection', 0, 255, nothing)
cv2.createTrackbar('High V', 'Object Detection', 255, 255, nothing)

while True:
    # Read the video frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Get the color thresholds from the trackbars
    low_h = cv2.getTrackbarPos('Low H', 'Object Detection')
    high_h = cv2.getTrackbarPos('High H', 'Object Detection')
    low_s = cv2.getTrackbarPos('Low S', 'Object Detection')
    high_s = cv2.getTrackbarPos('High S', 'Object Detection')
    low_v = cv2.getTrackbarPos('Low V', 'Object Detection')
    high_v = cv2.getTrackbarPos('High V', 'Object Detection')

    # Define the color thresholds
    lower_color = np.array([low_h, low_s, low_v])
    upper_color = np.array([high_h, high_s, high_v])

    # Threshold the HSV image to get only the object in the color range
    mask = cv2.inRange(hsv, lower_color, upper_color)

    # Apply morphological operations to remove noise
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.erode(mask, kernel)
    mask = cv2.dilate(mask, kernel)

    # Find contours of the object in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contours on the original frame
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 100:
            cv2.drawContours(frame, contour, -1, (0, 255, 0), 3)

    # Display the frames
    cv2.imshow('Object Detection', frame)
    cv2.imshow('Mask', mask)

    # Press 'q' to exit the program
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture and destroy all windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
### Mean Shift algorithm for object tracking in OpenCV:

import cv2

# Load the input video file
cap = cv2.VideoCapture('dog.mp4')

# Read the first frame from the input video
ret, frame = cap.read()

# Define the initial window location
x, y, w, h = 300, 200, 100, 100
track_window = (x, y, w, h)

# Extract the region of interest (ROI) from the first frame
roi = frame[y:y+h, x:x+w]

# Convert the ROI to the HSV color space
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Compute the histogram of the HSV color distribution in the ROI
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])

# Normalize the histogram
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Define the termination criteria for the Mean Shift algorithm
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

# Loop through the frames in the input video
while True:
    # Read a frame from the input video
    ret, frame = cap.read()

    # Stop the loop if the end of the video is reached
    if not ret:
        break

    # Convert the frame to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Calculate the back projection of the histogram onto the frame
    dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

    # Apply the Mean Shift algorithm to the back projection
    ret, track_window = cv2.meanShift(dst, track_window, term_crit)

    # Draw a rectangle around the tracked object
    x, y, w, h = track_window
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the output video frame with the rectangle
    cv2.imshow('Mean Shift Object Tracking', frame)

    # Exit on ESC key
    if cv2.waitKey(1) == 27:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
### CAMShift algorithm for object tracking in OpenCV:

import cv2
import numpy as np

# Load the input video file
cap = cv2.VideoCapture('dog.mp4')

# Read the first frame from the input video
ret, frame = cap.read()

# Define the initial window location
x, y, w, h = 300, 200, 100, 100
track_window = (x, y, w, h)

# Extract the region of interest (ROI) from the first frame
roi = frame[y:y+h, x:x+w]

# Convert the ROI to the HSV color space
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Compute the histogram of the HSV color distribution in the ROI
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])

# Normalize the histogram
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Define the termination criteria for the CAMShift algorithm
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

# Loop through the frames in the input video
while True:
    # Read a frame from the input video
    ret, frame = cap.read()

    # Stop the loop if the end of the video is reached
    if not ret:
        break

    # Convert the frame to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Calculate the back projection of the histogram onto the frame
    dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

    # Apply the CAMShift algorithm to the back projection
    ret, track_window = cv2.CamShift(dst, track_window, term_crit)

    # Draw an ellipse around the tracked object
    pts = cv2.boxPoints(ret)
    pts = np.int0(pts)
    cv2.polylines(frame, [pts], True, (0, 255, 0), 2)

    # Display the output video frame with the ellipse
    cv2.imshow('CAMShift Object Tracking', frame)

    # Exit on ESC key
    if cv2.waitKey(1) == 27:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



In [ ]:
### detecting markers with images using augmented reality (AR) in OpenCV.
# Note: Download  ArUco Markers images for the following exercise. 

import cv2
import numpy as np

# Load the dictionary and create the detector parameters
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

# Load the image 
img = cv2.imread("aruco_marker.png")

# Convert the frame to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect the markers in the frame
corners, ids, rejectedImgPoints = detector.detectMarkers(gray)

# Draw the detected markers and their IDs on the frame
if len(corners) > 0:
    ids = ids.flatten()
    for i, corner in enumerate(corners):
        cv2.aruco.drawDetectedMarkers(img, corners, ids)

# Show the frame
img = cv2.resize(img, (1600, 1200))
cv2.imshow('frame', img)


#close the function
cv2.waitKey(0)
# lose all windows
cv2.destroyAllWindows()


In [ ]:
### detecting markers with videos using augmented reality (AR) in OpenCV.

import cv2
import numpy as np

# Load the dictionary and create the detector parameters
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

# Start the camera capture
cap = cv2.VideoCapture(0)

while True:
    # Read the current frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect the markers in the frame
    corners, ids, rejectedImgPoints = detector.detectMarkers(gray)

    # Draw the detected markers and their IDs on the frame
    if len(corners) > 0:
        ids = ids.flatten()
        for i, corner in enumerate(corners):
            cv2.aruco.drawDetectedMarkers(frame, corners, ids)

    # Show the frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

